# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [20]:
# import numpy and pandas
import pandas as pd
import numpy as np
from statsmodels.stats.weightstats import ztest as ztest


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:

salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [7]:
# Your code here:
salaries.head(5)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [8]:
# Your code here:
salaries.isna().sum()


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [11]:
# Your code here:
salaries['Salary or Hourly'].value_counts()


Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [12]:
# Your code here:
salaries['Department'].value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [52]:
# Your code here:
hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']
hourly_rate = hourly['Hourly Rate']

from statsmodels.stats.weightstats import ztest as ztest
population_mean = hourly_rate.mean()
population_std = hourly_rate.std()
hourly_rate = list(hourly_rate)

_, p_value_hours = ztest(hourly_rate, value=30) #two-sided or smaller or greater
p_value_hours

# Since the p-value is lower than 0.05, we reject the null hypothesis that the wages are around 30$/h.

1.8228873859286195e-94

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [105]:
# Your code here:
salary = salaries[salaries['Salary or Hourly'] == 'Salary']
police_salary = salary[salary['Department'] == 'POLICE']
police_annual_salary = police_salary['Annual Salary']

_, p_value_police = ztest(police_annual_salary, value=86000, alternative='larger') #two-sided or smaller or greater
p_value_police

# Extremelly unlikely, we reject null hypothesis that the actual salaries are around 86k.


0.0010280845287547855

Using the `crosstab` function, find the department that has the most hourly workers. 

In [67]:
# Your code here:
my_crosstab = pd.crosstab(salaries['Salary or Hourly'], salaries['Department'])
my_crosstab

Department,ADMIN HEARNG,ANIMAL CONTRL,AVIATION,BOARD OF ELECTION,BOARD OF ETHICS,BUDGET & MGMT,BUILDINGS,BUSINESS AFFAIRS,CITY CLERK,CITY COUNCIL,...,MAYOR'S OFFICE,OEMC,POLICE,POLICE BOARD,PROCUREMENT,PUBLIC LIBRARY,STREETS & SAN,TRANSPORTN,TREASURER,WATER MGMNT
Salary or Hourly,,,,,,,,,,,,,,,,,,,,,
Hourly,0,19,1082,0,0,2,0,7,0,64,...,8,1273,10,0,2,299,1862,725,0,1513
Salary,39,62,547,107,8,44,269,164,84,347,...,77,829,13404,2,90,716,336,415,22,366


In [77]:
my_crosstab.loc['Hourly'].sort_values(ascending=False)[:3]

Department
STREETS & SAN    1862
WATER MGMNT      1513
OEMC             1273
Name: Hourly, dtype: int64

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [82]:
# Your code here:
streets = salaries[salaries['Department'] == 'STREETS & SAN']
streets_hourly = streets['Hourly Rate']
streets_hourly = streets_hourly.dropna()

import scipy

scipy.stats.ttest_1samp(streets_hourly, 35, alternative = 'less')

Ttest_1sampResult(statistic=-9.567447887848152, pvalue=1.6689265282353859e-21)

In [83]:
# We strongly reject the hypothesis that those guys get 35$ per hour!

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [ ]:
# Your code here:
# 0.95 CI, df.shape[0] - 1 degrees of freedom, sample mean for location parameter and std err for scale
# scipy.stats.sem(a, axis=0, ddof=1, nan_policy='propagate') for std err

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [109]:
# Your code here:
import scipy.stats as st

police = police_annual_salary
police_mean = np.mean(police)
police_std_err = st.sem(police)

CI_1 = st.t.interval(alpha=0.95, df=len(police)-1, loc=police_mean, scale=st.sem(police)) 

CI_2 = police_mean - ((1.96 * police_std_err )/ np.sqrt(len(police))), police_mean + ((1.96 * police_std_err) / np.sqrt(len(police)))

print(CI_1)
print(CI_2) # ?


(86177.05631531784, 86795.77269094894)
(86483.74264641479, 86489.086359852)


/var/folders/lr/stnkft6j5ps6pysrw7v0cgp80000gn/T/ipykernel_27978/536168454.py:8: DeprecationWarning: Use of keyword argument `alpha` for method `interval` is deprecated. Use first positional argument or keyword argument `confidence` instead.
  CI_1 = st.t.interval(alpha=0.95, df=len(police)-1, loc=police_mean, scale=st.sem(police))


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [115]:
# Your code here:

# statsmodels.stats.proportion.proportions_ztest(count, nobs, value=None, alternative='two-sided', prop_var=False)[source]¶
count = salaries[salaries['Salary or Hourly'] == 'Hourly'].shape[0]
nobs = salaries.shape[0]
value = 0.25



In [116]:
from statsmodels.stats.proportion import proportions_ztest

In [117]:
result = proportions_ztest(count, nobs, value)

In [118]:
result

(-3.5099964213703005, 0.0004481127249057967)

In [120]:
# We see that Chicago does not account for the 25% of the cases in the 95% confidence interval!
# p-value is 0.00045 approx.